In [ ]:
import pandas as pd
import numpy as np
from long_covid.colors import flatuicolors
from long_covid import styling
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from datetime import timedelta

# EQ5D

In [ ]:
tests = load('data/recent_tests.feather', True, pcr_tests)
vaccs = load('data/recent_vaccs.feather', True, vaccinations)
    
metadata = pd.merge(vaccs, tests, on='user_id')

In [ ]:
def load_eq5d(elements=(144, 145, 146, 147, 148)):
    
    if len(elements) == 1:
        formatter = '('+str(elements[0]) + ')'
    else:
        formatter = str(elements)
    
    query = """
    SELECT 
        answers.user_id, answers.questionnaire_session, answers.created_at, choice.choice_id 
    FROM 
        datenspende.answers, datenspende.choice 
    WHERE
        answers.element = choice.element and 
        answers.question in {0}
    """.format(formatter)

    db = postgre()
    
    df = pd.read_sql(query, db.conn)
    
    df = df.groupby(['user_id','questionnaire_session']).agg({'created_at': 'mean', 'choice_id': ['sum', 'count']})
    df.columns = df.columns.get_level_values(1)
    df['mean'] = pd.to_datetime(df['mean'], unit='ms').dt.date
    df.rename(columns={'mean': 'date', 'sum': 'EQ5D'}, inplace=True)
    df = df[df['count'] == len(elements)]
    df.drop(columns='count', inplace=True)
    
    return df.reset_index()

In [ ]:
LABEL = [
    'Overall health problems',
    'Problems in walking about',
    'Problems washing or dressing',
    'Problems in usual activities',
    'Pain or discomfort',
    'Anxiety and depression'
]   
    

f, axarr = plt.subplots(2, 3, figsize=(10, 6))

ELEMENTS = [(144, 145, 146, 147, 148), (144, ), (145, ), (146, ), (147, ), (148, )]

for element, ax, label in zip(ELEMENTS, axarr.flatten(), LABEL):

    eq5d = load_eq5d(elements=element)
    
    df = pd.merge(eq5d, metadata, on='user_id')
    df['EQ5D_trunc'] = round(df['EQ5D'] / len(element))
    
    #df = df[df.EQ5D_trunc > 1]
    #print(df.EQ5D_trunc.value_counts())

    ucount, x = np.histogram(df[df.user_id.isin(COHORTS['unvaccinated'])].EQ5D_trunc, bins=np.arange(0.5, 5.5))
    vcount, x = np.histogram(df[df.user_id.isin(COHORTS['vaccinated'])].EQ5D_trunc, bins=np.arange(0.5, 5.5))
    ncount, x = np.histogram(df[df.user_id.isin(COHORTS['negative'])].EQ5D_trunc, bins=np.arange(0.5, 5.5))
    
    print(ucount.sum(), vcount.sum(), ncount.sum())
    ucount = ucount / len(df[df.user_id.isin(COHORTS['unvaccinated'])])
    vcount = vcount / len(df[df.user_id.isin(COHORTS['vaccinated'])])
    ncount = ncount / len(df[df.user_id.isin(COHORTS['negative'])])
    
    x = x[:-1] + np.diff(x)[0] / 2
    width = np.diff(x)[0] / 3 * .7

    ax.bar(x-1.15 * width, ucount, width=width, label='Unvaccinated', color=flatuicolors.pomegranate)
    ax.bar(x, vcount, width=width, label='Vaccinated', color=flatuicolors.amethyst)
    ax.bar(x+1.15 * width, ncount, width=width, label='Negative', color=flatuicolors.belizehole)


    styling.hide_and_move_axis(ax)
    ax.set_xticks([1, 2, 3, 4, 5])
    ax.set_xticklabels(['None', 'Slight', 'Intermediate', 'Large', 'Extreme'], rotation=45, ha="right")
    
    ax.set_title(label)
    
for ax in axarr[:, 0]:
    ax.set_ylabel('Relative frequency')
    
ax.legend()
plt.tight_layout()
plt.savefig('output/eq5d.jpg', dpi=400)